# Exploring the Relationship Between Crime-Prone Wards and Residential Housing Prices in London

## 1. Introduction

### 1.1 Problem Statement

### 1.2 Research Question

## 2. Data Preparation

### 2.1 Data Origins

Table 1a of the ONS (2023) HPSSA 38 dataset, which provides quarterly data on home paid prices in the English and Walsh wards from December 1995 to March 2023, is the basis for the mean property prices using in the analysis. This time-series dataset includes 8059 observations with a total of 114 variables representing each quarter’s mean value.

Three Metropolitan Police Service datasets from London Datastore (2010; 2021; 2023) containing ward-level crime figures from 2001-2010, 2010-2021, and 2021-2023 formed the crime data basis. Hence, the first dataset comprises 116 variables representing the data time series and 22,178 observations accounting for all crime types by London ward. Similarly, there are 143 variables with 29,389 observations in the second, while the latter only has 29 variables and 23,498 observations.

### 2.2 Data Loading

In [3]:
pip install statsmodels

  Obtaining dependency information for statsmodels from https://files.pythonhosted.org/packages/7f/0a/28b3b3c807a518b7d0ed0ae45bfdd6be23b38a0c60790f90ce1f8383a640/statsmodels-0.14.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for patsy>=0.5.6 from https://files.pythonhosted.org/packages/43/f3/1d311a09c34f14f5973bb0bb0dc3a6e007e1eda90b5492d082689936ca51/patsy-0.5.6-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 101.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 44.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import libraries and modules needed for the entire project
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import TABLEAU_COLORS
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from matplotlib.ticker import StrMethodFormatter
import scipy.stats as stats
from scipy.stats import kruskal
from sklearn.preprocessing import PolynomialFeatures
import re
import matplotlib.colors as mcolors
from matplotlib.ticker import StrMethodFormatter
from PIL import Image
import statsmodels.api as sm
from statsmodels.robust.robust_linear_model import RLM

## 3. Data Cleaning & Manipulation

In [ ]:
# Steps to execute for crime data cleaning:
# Merge all data so that it is by monthyear (percolumn)
# Once merged, make the data a quarterly set, so that it is by march, june, sep, dec(per column)
# Group the crimes into wards - so we can see how much crime is happening in each ward, (cont')
# and also know which type of
# Group the crimes by ward, no need to detail the exact crimes, (cont'd)
# in the description we can talk about it more broadly. "each quarter, number of reported crimes are xxx.
# from our dataset, crimes include xxx"
# This gives a broader but clearer picture of number of crimes in each ward on a quarterly basis,
# using this to form heatmaps
# Make sure all my NaN values are null
# Remove 2001Q1 - 2007Q4 data from my dataset
# Address NaN values to ensure it is depicted as 0 instead.
# In report we might have to justify why we are using 0 to replace?

Merge crime data with ward

In [5]:
# Read CSVs
crime_data1 = pd.read_csv('MPS Ward Level Crime 2001-2010.csv')
crime_data2 = pd.read_csv('MPS Ward Level Crime 2010-2021.csv')
crime_data3 = pd.read_csv('MPS Ward Level Crime 2021-2023.csv')

# Preview initial data
crime_data1.head()

,WardCode,WardName,LookUp_BoroughName,MajorText,MinorText,200101,200102,200103,200104,200105,...,200906,200907,200908,200909,200910,200911,200912,201001,201002,201003
0,E05000026,Abbey,Barking and Dagenham,Violence Against the Person,ABH,5,15,5,8,9,...,0,0,0,0,0,0,0,0,0,0
1,E05000027,Alibon,Barking and Dagenham,Violence Against the Person,ABH,0,4,2,5,2,...,0,0,0,0,0,0,0,0,0,0
2,E05000028,Becontree,Barking and Dagenham,Violence Against the Person,ABH,4,3,1,4,10,...,0,0,0,0,0,0,0,0,0,0
3,E05000029,Chadwell Heath,Barking and Dagenham,Violence Against the Person,ABH,7,5,5,4,2,...,0,0,0,0,0,0,0,0,0,0
4,E05000030,Eastbrook,Barking and Dagenham,Violence Against the Person,ABH,2,3,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Merge all data so that it is by monthyear (per column)
# Merge on common columns throughout the 3 data sets
common_columns = ['WardName', 'WardCode', 'MajorText', 'MinorText', 'LookUp_BoroughName']

# An 'outer' join merges result to include all rows from both crime_data1 and crime_data2, (cont'd)
# Matching rows where possible based on the common columns and filling in NaNs where there is no match.
merged_crime_data = pd.merge(crime_data1, crime_data2, on=common_columns, how='outer')
merged_crime_data = pd.merge(merged_crime_data, crime_data3, on=common_columns, how='outer')

In [8]:
# Convert to quarterly data
def month_to_quarter(month):
    return (month - 1) // 3 + 1

quarterly_data = pd.DataFrame()
for col in merged_crime_data.columns:
    if col.isdigit() and len(col) == 6: # If the column represents a date
        year = col[:4]
        month = int(col[4:6])
        quarter = month_to_quarter(month)
        quarter_col = f'{year}Q{quarter}'
        
        if quarter_col not in quarterly_data:
            quarterly_data[quarter_col] = merged_crime_data[col]
        else:
            quarterly_data[quarter_col] += merged_crime_data[col]
    else:
        quarterly_data[col] = merged_crime_data[col]

In [9]:
# Preview data
quarterly_data.head()

,WardCode,WardName,LookUp_BoroughName,MajorText,MinorText,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,...,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3
0,E05000026,Abbey,Barking and Dagenham,Violence Against the Person,ABH,25.0,22.0,25.0,24.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E05000027,Alibon,Barking and Dagenham,Violence Against the Person,ABH,6.0,8.0,3.0,11.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E05000028,Becontree,Barking and Dagenham,Violence Against the Person,ABH,8.0,16.0,13.0,6.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E05000029,Chadwell Heath,Barking and Dagenham,Violence Against the Person,ABH,17.0,10.0,14.0,7.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E05000030,Eastbrook,Barking and Dagenham,Violence Against the Person,ABH,7.0,7.0,3.0,7.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Groupby operation on quarterly_data, grouping the data by LookUp_BoroughName, WardName, and WardCode.
# After grouping, sum of the grouped data is calculated, but only for the numeric columns
# Major and Minor crimes here cannot be merged, and every column is a different form of crime
grouped_by_ward = quarterly_data.groupby(['LookUp_BoroughName','WardName','WardCode']).sum(numeric_only=True)
grouped_by_ward.head()

2001Q1  2001Q2  2001Q3  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026   667.0   781.0   702.0   
                                           E05014053     0.0     0.0     0.0   
                         Alibon            E05000027   171.0   215.0   181.0   
                                           E05014054     0.0     0.0     0.0   

                                                      2001Q4  2002Q1  2002Q2  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026   710.0   770.0   732.0   
                                           E05014053     0.0     0.0     0.0   
                         Alibon            E05000027   177.0   196.0   213.0   
                                           E05014054     0.0     0.0     0.0   

                                                      2002Q3  2002Q4  2003Q1  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026   823.0   834.0   683.0   
                                           E05014053     0.0     0.0     0.0   
                         Alibon            E05000027   203.0   174.0   175.0   
                                           E05014054     0.0     0.0     0.0   

                                                      2003Q2  ...  2021Q2  \
LookUp_BoroughName       WardName          WardCode           ...           
Aviation Security (SO18) Heathrow Villages E05013570     0.0  ...     0.0   
Barking and Dagenham     Abbey             E05000026   664.0  ...     0.0   
                                           E05014053     0.0  ...   334.0   
                         Alibon            E05000027   235.0  ...     0.0   
                                           E05014054     0.0  ...   213.0   

                                                      2021Q3  2021Q4  2022Q1  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026     0.0     0.0     0.0   
                                           E05014053   356.0   383.0   398.0   
                         Alibon            E05000027     0.0     0.0     0.0   
                                           E05014054   224.0   200.0   227.0   

                                                      2022Q2  2022Q3  2022Q4  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026     0.0     0.0     0.0   
                                           E05014053   351.0   334.0   358.0   
                         Alibon            E05000027     0.0     0.0     0.0   
                                           E05014054   183.0   218.0   172.0   

                                                      2023Q1  2023Q2  2023Q3  
LookUp_BoroughName       WardName          WardCode                           
Aviation Security (SO18) Heathrow Villages E05013570     0.0   813.0  1379.0  
Barking and Dagenham     Abbey             E05000026     0.0     0.0     0.0  
                                           E05014053   421.0   404.0   440.0  
                         Alibon            E05000027     0.0     0.0     0.0  
                                           E05014054   176.0   213.0   180.0  

[5 rows x 91 columns]

### 3.2 Quantifying Crime

For ease of interpretation, we quantify crime by count within each ward. This allows us to have a consistent measure throughout our analysis when drawing comparisons between different wards.

In [11]:
# Replace NaN values with 0
grouped_by_ward.fillna(0, inplace=True)
grouped_by_ward.head()

2001Q1  2001Q2  2001Q3  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026   667.0   781.0   702.0   
                                           E05014053     0.0     0.0     0.0   
                         Alibon            E05000027   171.0   215.0   181.0   
                                           E05014054     0.0     0.0     0.0   

                                                      2001Q4  2002Q1  2002Q2  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026   710.0   770.0   732.0   
                                           E05014053     0.0     0.0     0.0   
                         Alibon            E05000027   177.0   196.0   213.0   
                                           E05014054     0.0     0.0     0.0   

                                                      2002Q3  2002Q4  2003Q1  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026   823.0   834.0   683.0   
                                           E05014053     0.0     0.0     0.0   
                         Alibon            E05000027   203.0   174.0   175.0   
                                           E05014054     0.0     0.0     0.0   

                                                      2003Q2  ...  2021Q2  \
LookUp_BoroughName       WardName          WardCode           ...           
Aviation Security (SO18) Heathrow Villages E05013570     0.0  ...     0.0   
Barking and Dagenham     Abbey             E05000026   664.0  ...     0.0   
                                           E05014053     0.0  ...   334.0   
                         Alibon            E05000027   235.0  ...     0.0   
                                           E05014054     0.0  ...   213.0   

                                                      2021Q3  2021Q4  2022Q1  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026     0.0     0.0     0.0   
                                           E05014053   356.0   383.0   398.0   
                         Alibon            E05000027     0.0     0.0     0.0   
                                           E05014054   224.0   200.0   227.0   

                                                      2022Q2  2022Q3  2022Q4  \
LookUp_BoroughName       WardName          WardCode                            
Aviation Security (SO18) Heathrow Villages E05013570     0.0     0.0     0.0   
Barking and Dagenham     Abbey             E05000026     0.0     0.0     0.0   
                                           E05014053   351.0   334.0   358.0   
                         Alibon            E05000027     0.0     0.0     0.0   
                                           E05014054   183.0   218.0   172.0   

                                                      2023Q1  2023Q2  2023Q3  
LookUp_BoroughName       WardName          WardCode                           
Aviation Security (SO18) Heathrow Villages E05013570     0.0   813.0  1379.0  
Barking and Dagenham     Abbey             E05000026     0.0     0.0     0.0  
                                           E05014053   421.0   404.0   440.0  
                         Alibon            E05000027     0.0     0.0     0.0  
                                           E05014054   176.0   213.0   180.0  

[5 rows x 91 columns]

In [12]:
# Check for NaN values
nan = grouped_by_ward.isnull().values.any()
# Output suggests data has no NaN values
nan

False

In [13]:
# Rename columns for better ease of join process with 'Mean House Price data'.
# Remove columns as our analysis only tracks from 2008Q1 - 2023Q1
# Generate a list of column names for quarters of years 2001 to 2007
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
columns_to_drop = [f'{year}{quarter}' for year in range(2001, 2008) for quarter in quarters]

# Drop these columns from the grouped_by_ward DataFrame
grouped_by_ward = grouped_by_ward.drop(columns = columns_to_drop, errors = 'ignore')

# Check the remaining columns to ensure the years 2001 to 2007 have been removed
print(grouped_by_ward.columns)

# This code creates a list of column names that include both the year and the quarter (e.g., '2001Q1', '2001Q2', etc.)
# The errors='ignore' parameter is used so that if any columns in columns_to_drop are not found in the DataFrame,
# it will not raise an error.

Index(['2008Q1', '2008Q2', '2008Q3', '2008Q4', '2009Q1', '2009Q2', '2009Q3',
       '2009Q4', '2010Q1', '2010Q2', '2010Q3', '2010Q4', '2011Q1', '2011Q2',
       '2011Q3', '2011Q4', '2012Q1', '2012Q2', '2012Q3', '2012Q4', '2013Q1',
       '2013Q2', '2013Q3', '2013Q4', '2014Q1', '2014Q2', '2014Q3', '2014Q4',
       '2015Q1', '2015Q2', '2015Q3', '2015Q4', '2016Q1', '2016Q2', '2016Q3',
       '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2018Q1', '2018Q2',
       '2018Q3', '2018Q4', '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1',
       '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4',
       '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3'],
      dtype='object')


In [14]:
grouped_by_ward.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1311 entries, ('Aviation Security (SO18)', 'Heathrow Villages', 'E05013570') to ('Westminster', 'Westbourne', 'E05013809')
Data columns (total 63 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2008Q1  1311 non-null   float64
 1   2008Q2  1311 non-null   float64
 2   2008Q3  1311 non-null   float64
 3   2008Q4  1311 non-null   float64
 4   2009Q1  1311 non-null   float64
 5   2009Q2  1311 non-null   float64
 6   2009Q3  1311 non-null   float64
 7   2009Q4  1311 non-null   float64
 8   2010Q1  1311 non-null   float64
 9   2010Q2  1311 non-null   float64
 10  2010Q3  1311 non-null   float64
 11  2010Q4  1311 non-null   float64
 12  2011Q1  1311 non-null   float64
 13  2011Q2  1311 non-null   float64
 14  2011Q3  1311 non-null   float64
 15  2011Q4  1311 non-null   float64
 16  2012Q1  1311 non-null   float64
 17  2012Q2  1311 non-null   float64
 18  2012Q3  1311 non-null   float64
 19  2012Q4  1311

In [15]:
# There is an issue with the cleaned dataset - there are duplicate WardNames with different WardCodes
# Merge the WardNames into 1, selecting Wardcodes based on the first WardCode to appear

# Check if 'WardCode' is a column
print('WardCode' in grouped_by_ward.columns)

# Check if 'WardCode' is in the index
print('WardCode' in grouped_by_ward.index.names)

False
True


In [16]:
# Reset index as WardCode needs to be a column
if 'WardCode' not in grouped_by_ward.columns and 'WardCode' in grouped_by_ward.index.names:
    grouped_by_ward.reset_index(inplace = True)

# Check if 'WardCode' is a column
print('WardCode' in grouped_by_ward.columns)

# Check if 'WardCode' is in the index
print('WardCode' in grouped_by_ward.index.names)

True
False


In [19]:
# Step 1: Aggregate data while keeping 'LookUp_BoroughName' and 'WardCode'
first_ward_code = grouped_by_ward.groupby('WardName')['WardCode'].first()
borough_name_for_ward = grouped_by_ward.groupby('WardName')['LookUp_BoroughName'].first()

# Aggregate the numeric data
numeric_cols = [col for col in grouped_by_ward.columns if grouped_by_ward[col].dtype != 'object']
grouped_ward = grouped_by_ward.groupby('WardName')[numeric_cols].sum()

# Step 2: Add 'WardCode' and 'LookUp_BoroughName' back to the DataFrame
grouped_ward['WardCode'] = first_ward_code.values
grouped_ward['LookUp_BoroughName'] = borough_name_for_ward.values

# Reset the index to turn 'WardName' back into a column
grouped_ward.reset_index(inplace=True)

# Step 3: Reorder the columns
columns_order = ['LookUp_BoroughName', 'WardName', 'WardCode'] + [col for col in grouped_ward.columns if col not in ['LookUp_BoroughName', 'WardName', 'WardCode']]
grouped_ward = grouped_ward[columns_order]

# Sort the rows based on 'LookUp_BoroughName'
grouped_ward = grouped_ward.sort_values(by = 'LookUp_BoroughName')

# Verify the results
print(grouped_ward.head())

           LookUp_BoroughName           WardName   WardCode  2008Q1  2008Q2  \
380  Aviation Security (SO18)  Heathrow Villages  E05013570   600.0   715.0   
0        Barking and Dagenham              Abbey  E05000026   828.0   863.0   
574      Barking and Dagenham           Parsloes  E05000037   205.0   233.0   
497      Barking and Dagenham         Mayesbrook  E05000036   239.0   260.0   
479      Barking and Dagenham         Longbridge  E05000035   188.0   192.0   

     2008Q3  2008Q4  2009Q1  2009Q2  2009Q3  ...  2021Q2  2021Q3  2021Q4  \
380   935.0   938.0   906.0   875.0   826.0  ...   649.0   833.0  1029.0   
0     836.0   842.0   946.0   902.0   870.0  ...   483.0   534.0   555.0   
574   209.0   181.0   208.0   209.0   201.0  ...   287.0   243.0   258.0   
497   231.0   240.0   255.0   244.0   259.0  ...   259.0   238.0   225.0   
479   205.0   167.0   200.0   212.0   198.0  ...   214.0   199.0   259.0   

     2022Q1  2022Q2  2022Q3  2022Q4  2023Q1  2023Q2  2023Q3  
380  1

Save to CSV:

In [20]:
# Save to CSV
grouped_ward.to_csv('final_cleaned_crime_data.csv', index=False)

In [21]:
# There are certain columns in the dataset that is 0, not recorded, we do not have information on this
# to address in our analysis

### 3.3 Data Cleaning & Manipulation for Mean House Prices

In [22]:
# Read CSV and preview
ward_file = pd.read_csv('ward_price.csv')
ward_file

,Local authority code,Local authority name,Ward code,Ward name,Year ending Dec 1995,Year ending Mar 1996,Year ending Jun 1996,Year ending Sep 1996,Year ending Dec 1996,Year ending Mar 1997,...,Year ending Dec 2020,Year ending Mar 2021,Year ending Jun 2021,Year ending Sep 2021,Year ending Dec 2021,Year ending Mar 2022,Year ending Jun 2022,Year ending Sep 2022,Year ending Dec 2022,Year ending Mar 2023
0,E06000001,Hartlepool,E05008945,Foggy Furze,"38,358","39,211","39,527","38,166","37,390","37,089",...,"106,917","110,952","116,335","119,909","121,248","120,448","112,693","111,655","110,292","110,895"
1,E06000001,Hartlepool,E05008946,Hart,"57,172","57,227","58,418","58,647","64,780","67,035",...,"177,508","178,773","184,947","186,662","183,006","188,158","184,160","185,957","194,018","198,887"
2,E06000001,Hartlepool,E05008947,Headland and Harbour,"32,604","33,855","35,144","33,417","33,376","31,733",...,"102,259","103,893","118,331","123,295","129,184","132,543","126,461","123,524","126,299","128,285"
3,E06000001,Hartlepool,E05008943,De Bruce,"42,074","42,164","39,192","35,915","35,119","34,856",...,"142,404","143,566","141,864","137,110","136,954","129,942","127,852","132,312","129,913","136,445"
4,E06000001,Hartlepool,E05008944,Fens and Rossmere,"51,959","51,206","49,653","47,954","47,533","47,779",...,"136,584","139,956","142,371","146,607","150,250","151,938","152,607","153,394","158,404","158,017"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8048,W06000024,Merthyr Tydfil,W05000895,Vaynor,"48,505","47,831","47,823","50,143","45,945","46,191",...,"149,431","195,417","195,333","200,137","182,961","175,638","184,341","192,031","212,074","220,955"
8049,W06000024,Merthyr Tydfil,W05000707,Bedlinog,"25,799","24,203","23,338","23,948","24,442","30,155",...,"131,200","153,231","180,903","173,038","175,144","170,973","151,312","159,547","151,936","159,457"
8050,W06000024,Merthyr Tydfil,W05000708,Cyfarthfa,"44,344","45,043","47,203","46,270","45,764","45,185",...,"177,794","180,401","187,092","194,152","189,072","194,875","199,416","210,262","221,021","226,777"
8051,W06000024,Merthyr Tydfil,W05000710,Gurnos,"33,166","32,255","29,661","29,297","33,497","37,738",...,"131,316","180,667","181,778","148,750","157,632","122,577","126,346","120,317","128,083","138,000"


In [23]:
# Drop columns that are not needed as we are only tracking from 2008Q1 to 2023Q1
# Check for null values
ward_file.iloc[:,53].info()
ward_file.drop(ward_file.iloc[:,4:53],axis = 1,inplace = True)

<class 'pandas.core.series.Series'>
RangeIndex: 8053 entries, 0 to 8052
Series name: Year ending Mar 2008
Non-Null Count  Dtype 
--------------  ----- 
8053 non-null   object
dtypes: object(1)
memory usage: 63.0+ KB


In [24]:
ward_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8053 entries, 0 to 8052
Data columns (total 65 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Local authority code  8053 non-null   object
 1   Local authority name  8053 non-null   object
 2   Ward code             8053 non-null   object
 3   Ward name             8053 non-null   object
 4   Year ending Mar 2008  8053 non-null   object
 5   Year ending Jun 2008  8053 non-null   object
 6   Year ending Sep 2008  8053 non-null   object
 7   Year ending Dec 2008  8053 non-null   object
 8   Year ending Mar 2009  8053 non-null   object
 9   Year ending Jun 2009  8053 non-null   object
 10  Year ending Sep 2009  8053 non-null   object
 11  Year ending Dec 2009  8053 non-null   object
 12  Year ending Mar 2010  8053 non-null   object
 13  Year ending Jun 2010  8053 non-null   object
 14  Year ending Sep 2010  8053 non-null   object
 15  Year ending Dec 2010  8053 non-null   

In [25]:
# Original dataset covers the entire UK, we filtered for boroughs in London only
london_borough_names = ['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich',
'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets',
'Waltham Forest', 'Wandsworth', 'Westminster']
new_file = ward_file[ward_file['Local authority name'].isin(london_borough_names)]
new_file

,Local authority code,Local authority name,Ward code,Ward name,Year ending Mar 2008,Year ending Jun 2008,Year ending Sep 2008,Year ending Dec 2008,Year ending Mar 2009,Year ending Jun 2009,...,Year ending Dec 2020,Year ending Mar 2021,Year ending Jun 2021,Year ending Sep 2021,Year ending Dec 2021,Year ending Mar 2022,Year ending Jun 2022,Year ending Sep 2022,Year ending Dec 2022,Year ending Mar 2023
6569,E09000002,Barking and Dagenham,E05000026,Abbey,"183,504","185,865","188,033","180,628","177,795","177,714",...,"309,509","312,336","310,881","304,485","290,932","284,982","267,432","253,508","262,483","260,524"
6570,E09000002,Barking and Dagenham,E05000027,Alibon,"186,749","189,167","190,669","190,602","183,746","173,878",...,"300,085","304,197","307,583","320,590","322,404","332,452","336,013","335,426","354,199","349,081"
6571,E09000002,Barking and Dagenham,E05000033,Goresbrook,"192,907","187,727","183,496","179,072","167,569","155,869",...,"296,574","296,298","305,796","307,588","312,030","328,704","339,094","353,588","364,151","360,784"
6572,E09000002,Barking and Dagenham,E05000030,Eastbrook,"214,167","215,915","210,896","212,248","205,985","197,470",...,"359,260","371,028","382,167","395,369","404,315","406,768","408,359","414,049","408,422","406,554"
6573,E09000002,Barking and Dagenham,E05000036,Mayesbrook,"190,229","192,777","189,357","194,160","186,768","176,199",...,"315,524","316,809","331,721","339,602","347,399","366,255","363,058","375,442","371,311","369,390"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7196,E09000033,Westminster,E05000645,Tachbrook,"586,338","544,712","523,906","547,769","557,697","606,388",...,"935,824","972,679","959,718","982,261","965,033","930,908","922,101","1,024,718","1,027,889","1,025,746"
7197,E09000033,Westminster,E05000648,Westbourne,"382,275","396,129","384,627","421,463","411,105","417,419",...,"716,765","732,861","834,021","846,247","831,008","837,721","706,903","743,713","744,704","730,266"
7198,E09000033,Westminster,E05000649,West End,"1,031,714","1,077,817","1,057,118","1,008,513","1,037,526","897,517",...,"5,548,134","5,147,406","4,377,806","3,282,339","3,572,462","4,127,796","4,828,008","4,891,753","4,598,264","3,710,331"
7199,E09000033,Westminster,E05000647,Warwick,"669,221","664,196","654,705","653,299","637,444","597,295",...,"1,124,086","1,062,568","1,067,681","1,071,415","1,044,036","1,088,682","1,280,840","1,323,287","1,299,517","1,287,754"


In [26]:
# Save to CSV
new_file.to_csv('filtered_london_prices.csv', index = False)

In [ ]:
# CLEANING OF HOUSE PRICE DATA ENDS HERE, START DATA MANIPULATION & MERGING FOR FINAL SET HERE

### 3.4 Merging the Datasets

In [ ]:
# Clean house price data - (1) remove local authority code from filtered_london_prices data
# Clean house price data - (2) make the house price values consistent, rounding all values to 2 decimal points(float)

# Change the 'Year ending Month Year' rows to 2008Q1 - 2023Q3
# Merge datasets based on names LookUp_BoroughName, WardName, WardCode
# For merged sets, we want each quarter row to show 2 information - house price, followed by n number of crime.